In [34]:
from IPython.core.debugger import set_trace
import pickle
import numpy as np
import matplotlib.pyplot as plt

# pre-requirement: CIFAR-10 package is downloaded and unpackaged 
# from http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
class Cifar10Input:
      
    def __init__(self, data, folder="../convolution-network"):
      
        self.classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"] 
        fdata =["data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4", "data_batch_5"]
        ftest = "test_batch"
        n_row = n_col = 32
        n_depth = 3
        
        folder += "/cifar-10-batches-py/"
        temp = []
        X = Y = None
        if data == "train":
            for name in fdata:
                fname = folder + name
                with open(fname, 'rb') as f:
                    dict = pickle.load(f, encoding='bytes')
                    X = np.array(dict[b"data"])
                    Y = np.array(dict[b"labels"])
                    X = X.reshape(-1, n_depth, n_row, n_col)
                    temp.append((X, Y))
            
        else:
            assert data == "test"
            fname = folder + ftest
            with open(fname, 'rb') as f:
                dict = pickle.load(f, encoding='bytes')
                X = np.array(dict[b"data"])
                Y = np.array(dict[b"labels"])
                X = X.reshape(-1, n_depth, n_row, n_col)
                temp.append((X, Y))
                
        self.data = temp
        
        return
    
    def read_sample(self, n_sample, nth_batch=0):
        n_batches = len(self.data)
        assert nth_batch >= 0 and nth_batch < n_batches

        X, Y = self.data[nth_batch]
        if n_sample == -1: n_sample = len(X)
        for i in range(n_sample):
            yield (X[i], Y[i])
        
        return
        
    def read_batch(self, nth_batch=0):
        n_batches = len(self.data)
        assert (nth_batch >= 0 and nth_batch < n_batches) or nth_batch == -1 
        # if nth_batch = -1, to read all batches
        for i in range(n_batches):
            if i != nth_batch and nth_batch != -1: continue
            for x, y in self.read_sample(-1, i):
                yield (x, y)
                                
        return


#cifar_test = Cifar10Input("test")
#for x, y in cifar_test.read_sample(10):
    #print(cifar_test.classes[y])
    ##from scipy.misc import toimage    
    ##plt.imshow(toimage(x))
    #plt.imshow(x.transpose(1,2,0))
    #plt.show()


In [35]:
class CIFAR10:

    def __init__(self, nn, folder="../convolution-network"):   
        self.nn = nn
        self.train_input = Cifar10Input("train", folder)
        self.test_input = Cifar10Input("test", folder)
        return
    
    def train(self, nth_batch=-1):
        i = 1
        for X, y in self.train_input.read_batch(nth_batch):
            X = X/255
            if self.nn.type == "MLP":
                X = X.reshape(-1,1)                

            self.nn.train_1sample(X, y)
        return
            
    def test(self, nth_batch=0):
        correct = 0
        total = 0
        for X, y in self.test_input.read_batch(nth_batch):
            aX = X/255
            if self.nn.type == "MLP":
                aX = aX.reshape(-1,1)

            predict = self.nn.predict_1sample(aX)
            correct += 1 * (predict == y)
            total += 1
            #print("\nPredict {} to be {}:".format(y, predict))
            #plt.imshow(X, cmap=mpl.cm.Greys)
            #plt.show()
            
        accuracy = correct/total
        return accuracy